In [ ]:
import os
import datetime
from pathlib import Path
import glob
import itertools

folders = glob.glob(r"example")
print(folders)
for folder in folders:
    # folder = "data\\2024-10-03\\2 VLP +only L"
    transform_matrix = r"transform_matrix.json"
    # brightfield_transform_matrix = r"bf_transform_matrix.json"
    separate_folders = False
    normalize = False
    ignore_framerate = True  # This should be false usually. There was an issue with the framerates in some videos Vid took and this solves that problem
    include_empty_channel = False
    timestamps = []
    tiff_filelist = glob.glob(f"{folder}/*.tif*")
    output_path = os.path.join(folder, "aligned_output")
    # output_path = "output_all_colors"
    print(output_path)

    def roundTime(dt=None, roundTo=60):
        """Round a datetime object to any time lapse in seconds
        dt : datetime.datetime object, default now.
        roundTo : Closest number of seconds to round to, default 1 minute.
        Author: Thierry Husson 2012 - Use it as you want but don't blame me.
        """
        if dt == None:
            dt = datetime.datetime.now()
        seconds = (dt.replace(tzinfo=None) - dt.min).seconds
        rounding = (seconds + roundTo / 2) // roundTo * roundTo
        return dt + datetime.timedelta(0, rounding - seconds, -dt.microsecond)

    def get_timestamp(filename):
        round_to_seconds = 60
        time = roundTime(
            datetime.datetime.fromtimestamp(os.path.getmtime(filename)),
            round_to_seconds,
        )
        # print(time)
        return time

    def get_filename_no_ending(filename):
        filename_no_ending = os.path.basename(filename).split("_")[0]
        return filename_no_ending

    tiff_filelist_name = sorted(tiff_filelist, key=get_timestamp)
    wt_irm = itertools.groupby(tiff_filelist_name, get_timestamp)
    # print(tiff_filelist_name)
    grouped_by_time = []
    for key, group in wt_irm:
        listgroup = list(group)
        if len(listgroup) == 2:
            grouped_by_time.append(listgroup)
        else:
            print(f"A group found with {len(listgroup)} members")
    print("grouped by time")
    print(grouped_by_time)

    tiff_filelist_time = sorted(tiff_filelist, key=get_filename_no_ending)
    wt_irm = itertools.groupby(tiff_filelist_time, get_filename_no_ending)
    grouped_by_name = []
    for key, group in wt_irm:
        listgroup = list(group)
        if len(listgroup) == 2:
            grouped_by_name.append(listgroup)
    print("grouped by name")
    print(grouped_by_name)

    all_grouped = grouped_by_name + grouped_by_time
    all_grouped.sort()
    all_grouped = list(
        all_grouped for kall_grouped, _ in itertools.groupby(all_grouped)
    )[0]
    print("----")
    for group in all_grouped:
        print(group)
        if len(group) == 2:
            for file in group:
                if "_IRM" in file:
                    irm_file = file
                if "_WT" in file:
                    wt_file = file
            assert "_IRM" in irm_file, "no IRM file"
            assert "_WT" in wt_file, "no WT file"
            if separate_folders:
                subdirectory = os.path.join(
                    output_path, os.path.splitext(os.path.basename(wt_file))[0]
                )
                print(subdirectory)
            else:
                subdirectory = output_path
            cmd_string = f'python video_aligner.py "{wt_file}" "{irm_file}" -m "{transform_matrix}" -o "{subdirectory}" '
            if normalize:
                cmd_string += " -n"
            if include_empty_channel:
                cmd_string += " -ec"
            if ignore_framerate:
                cmd_string += " -if"
            print(cmd_string)
            os.system(cmd_string)
        else:
            print(f"{len(group)} files were found in the same group")

    """ for file in tiff_filelist:
        time = datetime.datetime.fromtimestamp(os.path.getmtime(file)).replace(
            second=0, microsecond=0
        )
        if time not in timestamps:
            timestamps.append(time)

    for timestamp in timestamps:
        for file in tiff_filelist:
            time = datetime.datetime.fromtimestamp(os.path.getmtime(file)).replace(
                second=0, microsecond=0
            )
            if time == timestamp:
                if "WT" in file:
                    wt_file = file
                else:
                    irm_file = file 

        output_path = os.path.join(folder, Path(wt_file).stem + "_output")

        print(f"!python image_aligner.py {wt_file} {irm_file} -m '{transform_matrix}' -o {output_path}") """